In [1]:
import subprocess
import re
from Dtest_randomizer import generate_agents_on_map
import time as timer
import os

def run_terminal_test(test_map_filename, solver_name):
    command = ["python", "run_experiments.py", "--instance", f"instances/{test_map_filename}", "--solver", f"{solver_name}"]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    return result.stdout, result.stderr

def monte_carlo_simulation(num_iterations, test_map_filename, solver_name_list, new_map_generation, input_file_path, output_file_path, num_agents, type = "all"):
    '''Monte Carlo simulation for a given solver and a given number of iterations. '''
    results = []
    average_cost = []
    average_makespan = []
    average_cpu_time = []
    missing_calculations = 0
    
    start_time = timer.time()
    os.makedirs(f"MC_results", exist_ok=True)

    for solver_name in (solver_name_list):
        print(f'Current solver: {solver_name}')
        # Create a directory for the solver if it doesn't already exist
        with open(f'MC_results/MC_results_{solver_name}_iterations_{num_iterations}.txt', 'a') as f:
            f.write(f"Results for {solver_name} with {num_iterations} iterations:\n")

        for i in range(num_iterations):
            try:
                if new_map_generation == True:
                    generate_agents_on_map(input_file_path, output_file_path, num_agents, i + 1)        # Generate a new map for each iteration

                stdout, stderr = run_terminal_test(test_map_filename, solver_name)                      # Run the solver in the terminal
            
                cpu_time_pattern = r"CPU time \(s\):\s*(\S+)"                                           # Extract the CPU time, sum of costs makespan, and goal priority from the terminal output
                sum_of_costs_pattern = r"Sum of costs:\s*(\d+)"
                makespan_pattern = r"Makespan:\s*(\d+) \(agent \d+\)"
                goal_priority_pattern = r"goal priority:\s*(\w+)"

                cpu_time = re.search(cpu_time_pattern, stdout)                                          # Search for the patterns mentioned above in the terminal output                     
                sum_of_costs = re.search(sum_of_costs_pattern, stdout)
                makespan = re.search(makespan_pattern, stdout)
                goal_priority = re.search(goal_priority_pattern, stdout)

                if sum_of_costs and makespan and cpu_time:
                    sum_of_costs = sum_of_costs.group(1)                                                # Extract the values from the regex search             
                    makespan_unsplit = makespan.group(0)
                    cpu_time = cpu_time.group(1)                                                        
                    goal_priority = goal_priority.group(1) if goal_priority else None                   # If the goal priority is not found, set it to None

                    result = f"Cost: {sum_of_costs}, {makespan_unsplit}, CPU time: {cpu_time}, Goal priority: {goal_priority}, iteration: {i + 1}"      # Create a string with the results
                    print(result)

                    with open(f'MC_results/MC_results_{solver_name}_iterations_{num_iterations}.txt', 'a') as f:
                        f.write(f"{result}\n")

                    average_cost.append(int(sum_of_costs))
                    parts = makespan_unsplit.split(':')
                    makespan = int(parts[1].split()[0])
                    average_makespan.append(int(makespan))
                    average_cpu_time.append(float(cpu_time))

                    results.append(result)

                else:
                    missing_calculations += 1
                    print(f"Missing data in iteration {i + 1}")

                    with open(f'MC_results/MC_results_{solver_name}_iterations_{num_iterations}.txt', 'a') as f:
                        f.write(f"Missing data in iteration {i + 1}\n")


            except Exception as e:
                print(f"An error occurred in iteration {i}: {e}")
                with open(f'MC_results/MC_results_{solver_name}_iterations_{num_iterations}.txt', 'a') as f:
                    f.write(f"An error occurred in iteration {i}: {e}\n")

            print(f"Iteration {i + 1} completed")


    CPU_time = timer.time() - start_time

    if average_cost and average_makespan and average_cpu_time:
        average_cost = round(sum(average_cost) / len(average_cost), 2)
        average_makespan = round(sum(average_makespan) / len(average_makespan), 2)
        average_cpu_time = round(sum(average_cpu_time) / len(average_cpu_time), 2)

        with open(f"MC_results/MC_results_{solver_name}_iterations_{num_iterations}_avg_results.txt", 'a') as file:
                file.write(f"The Monte Carlo Simulation for {solver_name} with {num_iterations} iterations has the following results:\n"
                           f"Average cost: {average_cost}\n"
                           f"Average makespan: {average_makespan}\n"
                           f"Average CPU time: {average_cpu_time}\n"
                           f"Total Monte Carlo CPU time: {round(CPU_time, 2)}\n"
                           f"Missing calculations: {missing_calculations}\n")
    else:
        print("No valid data to calculate averages.")

    return results

input_file_path = "instances\Dtest_situatie_3.txt"
output_file_path = "instances\Dtest_function_updated.txt"

solve_name_list_all = ["CBShigher", "CBSfalse", "Distributedtrue", "Distributedhigher", "Distributedfalse", "Prioritized", "CBSlower", "Distributedlower", "CBStrue"]

nr_of_runs = 100
nr_of_agents = 10
monte_carlo_simulation(nr_of_runs, "Dtest_function_updated.txt", solve_name_list_all, True, input_file_path, output_file_path, nr_of_agents)

Current solver: CBShigher
Cost: 132, Makespan:        25 (agent 7), CPU time: 40.67, Goal priority: higher, iteration: 1
Iteration 1 completed
Cost: 99, Makespan:        25 (agent 7), CPU time: 56.05, Goal priority: higher, iteration: 2
Iteration 2 completed
Cost: 149, Makespan:        29 (agent 4), CPU time: 79.55, Goal priority: higher, iteration: 3
Iteration 3 completed
Cost: 88, Makespan:        18 (agent 6), CPU time: 39.02, Goal priority: higher, iteration: 4
Iteration 4 completed
Missing data in iteration 4
Iteration 5 completed
Missing data in iteration 5
Iteration 6 completed
Cost: 113, Makespan:        19 (agent 5), CPU time: 27.09, Goal priority: higher, iteration: 7
Iteration 7 completed
Cost: 129, Makespan:        23 (agent 1), CPU time: 13.68, Goal priority: higher, iteration: 8
Iteration 8 completed
Cost: 139, Makespan:        24 (agent 3), CPU time: 19.73, Goal priority: higher, iteration: 9
Iteration 9 completed
Cost: 144, Makespan:        28 (agent 0), CPU time: 14.14

['Cost: 132, Makespan:        25 (agent 7), CPU time: 40.67, Goal priority: higher, iteration: 1',
 'Cost: 99, Makespan:        25 (agent 7), CPU time: 56.05, Goal priority: higher, iteration: 2',
 'Cost: 149, Makespan:        29 (agent 4), CPU time: 79.55, Goal priority: higher, iteration: 3',
 'Cost: 88, Makespan:        18 (agent 6), CPU time: 39.02, Goal priority: higher, iteration: 4',
 'Cost: 113, Makespan:        19 (agent 5), CPU time: 27.09, Goal priority: higher, iteration: 7',
 'Cost: 129, Makespan:        23 (agent 1), CPU time: 13.68, Goal priority: higher, iteration: 8',
 'Cost: 139, Makespan:        24 (agent 3), CPU time: 19.73, Goal priority: higher, iteration: 9',
 'Cost: 144, Makespan:        28 (agent 0), CPU time: 14.14, Goal priority: higher, iteration: 10',
 'Cost: 98, Makespan:        24 (agent 7), CPU time: 25.19, Goal priority: higher, iteration: 11',
 'Cost: 122, Makespan:        19 (agent 0), CPU time: 28.55, Goal priority: higher, iteration: 12',
 'Cost: 15